### **EDA with SQL**

#### **Introduction**
Using this Python notebook we will:

* Understand the Spacex DataSet
* Load the dataset into the corresponding table in a Db2 database
* Execute SQL queries to answer assignment questions

#### **Import Libraries**

Let us first load the SQL extension and establish a connection with the database. In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. 


In [20]:
!pip install ipython-sql
!pip install ipython-sql prettytable
!pip install sqlalchemy==1.3.9


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ----- ---------------------------------- 0.8/6.0 MB 7.6 MB/s eta 0:00:01
     ------- -------------------------------- 1.0/6.0 MB 2.7 MB/s eta 0:00:02
     -------------- ------------------------- 2.1/6.0 MB 3.5 MB/s eta 0:00:02
     ------------------------ --------------- 3.7/6.0 MB 4.6 MB/s eta 0:00:01
     --------------------------------- ------ 5.0/6.0 MB 5.1 MB/s eta 0:00:01
     ---------------------------------------- 6.0/6.0 MB 5.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sqlalchemy: filename=sqlalchemy-1.3.9-cp313-cp313-win_amd64.whl size=1143040 sha256=624edf147bc0986685e6e23fdf6

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
%load_ext sql

import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [23]:
import pandas as pd
df = pd.read_csv("EDA_SQL.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

#### **Tasks**

Display the names of the unique launch sites in the space mission

In [24]:
%%sql
SELECT DISTINCT Launch_Site
FROM SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Display 5 records where launch sites begin with the string 'CCA'

In [25]:
%%sql 
SELECT *
FROM SPACEXTBL
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Display the total payload mass carried by boosters launched by NASA (CRS)

In [27]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) AS TotalPayloadMass
FROM SPACEXTBL
WHERE Customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


TotalPayloadMass
45596


Display average payload mass carried by booster version F9 v1.1

In [28]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS AvgPayloadMass
FROM SPACEXTBL
WHERE Booster_Version = 'F9 v1.1'


 * sqlite:///my_data1.db
Done.


AvgPayloadMass
2928.4


List the date when the first succesful landing outcome in ground pad was acheived.

In [29]:
%%sql
SELECT MIN(Date) AS FirstLandingDate
FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


FirstLandingDate
2015-12-22


List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [30]:
%%sql
SELECT DISTINCT Booster_version 
FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (drone ship)'
AND PAYLOAD_MASS__KG_ between 4000 AND 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


List the total number of successful and failure mission outcomes

In [31]:
%%sql
SELECT
  Mission_Outcome,
  COUNT(*) AS Total
FROM SPACEXTBL
GROUP BY Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Total
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.

In [32]:
%%sql
SELECT DISTINCT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (
  SELECT MAX(PAYLOAD_MASS__KG_)
  FROM SPACEXTBL
)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [33]:
%%sql 
SELECT
  CASE substr(Date,6,2)
    WHEN '01' THEN 'January'
    WHEN '02' THEN 'February'
    WHEN '03' THEN 'March'
    WHEN '04' THEN 'April'
    WHEN '05' THEN 'May'
    WHEN '06' THEN 'June'
    WHEN '07' THEN 'July'
    WHEN '08' THEN 'August'
    WHEN '09' THEN 'September'
    WHEN '10' THEN 'October'
    WHEN '11' THEN 'November'
    WHEN '12' THEN 'December'
  END AS Month,
  Landing_Outcome,
  Booster_Version,
  Launch_Site
FROM SPACEXTBL
WHERE substr(Date,1,4) = '2015'
  AND Landing_Outcome = 'Failure (drone ship)'

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [34]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS outcome_count
FROM SPACEXTBL
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY outcome_count DESC

 * sqlite:///my_data1.db
Done.


Landing_Outcome,outcome_count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
